In [32]:
![ -f ../bin/chrome-linux64 ] \
    || wget -qq -O ../bin/chrome-linux64.zip https://storage.googleapis.com/chrome-for-testing-public/114.0.5735.90/linux64/chrome-linux64.zip \
    && unzip -q ../bin/chrome-linux64.zip -d ../bin \
    && rm ../bin/chrome-linux64.zip

In [3]:
import bs4
import httpx

from selenium import webdriver

In [5]:
driver = webdriver.Chrome(executable_path="../bin/chrome-linux64/")
driver.get("https://www.flashscore.com/team/germany/ptQide1O/results/");
time.sleep(5) # Let the user actually see something!

TypeError: WebDriver.__init__() got an unexpected keyword argument 'executable_path'

In [ ]:
search_box = driver.find_element_by_name('q')
search_box.send_keys('ChromeDriver')
search_box.submit()
time.sleep(5) # Let the user actually see something!
driver.quit()

In [3]:
res = httpx.get("https://www.flashscore.com/team/germany/ptQide1O/results/")

In [4]:
soup = bs4.BeautifulSoup(res.text)

In [5]:
results = soup.find("div", class_="leagues--static event--leagues results")

In [11]:
with open("index.html", "w") as f:
    f.write(str(soup))

In [7]:
results